<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

Параметры подключения из юнита 1. Работа с базой данных из Python.

In [2]:
connection = psycopg2.connect(
    dbname='project_sql',
    user='skillfactory',
    host='84.201.134.129',
    password='cCkxxLVrDE8EbvjueeMedPKt',
    port=5432
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
query_3_1 = f'''
    select
	    count(*)
    from 
        public.vacancies
    '''

df = pd.read_sql_query(query_3_1,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [4]:
query_3_2 = f'''
    select
	    count(distinct id)
    from 
        public.employers
    '''

df = pd.read_sql_query(query_3_2,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [5]:
query_3_3 = f'''
    select
	    count(*)
    from 
        public.areas
    '''

df = pd.read_sql_query(query_3_3,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [6]:
query_3_4 = f'''
    select
	    count(*)
    from 
        public.industries
    '''

df = pd.read_sql_query(query_3_4,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,294


***

__Выводы по предварительному анализу данных:__ С помощью предварительного анализа данных мы пришли к выводу, что интересующие данные представлены в пяти таблицах, из которых одна является основной, непосредственно содержащей данные о вакансиях (vacancies), а четыре остальных имеют справочный характер и хранят информацию о всех работодателях, сферах деятельности, связях работодателей и сфер деятельности, территориях (employers, industries, emloyers_industries, areas соответственно).
В таблицах представлено 49197 вакансий, 23905 работодателей, 294 сферы деятельности, 1362 региона. Стоит отметить, что в таблице vacancies присутствуют неуникальные работодатели и регионы.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [7]:
query_4_1 = f'''
    select
	    a.name area,
	    count(v.id) cnt /* Подсчет числа вакансий в каждом регионе */
    from 
        public.vacancies v
	    left join public.areas a on v.area_id = a.id /* Соединение таблиц вакансий и территорий по id террритории */
    group by area /* Группировка по территориям */
    order by cnt desc /* Сортировка по числу вакансий по убыванию */
    '''

df = pd.read_sql_query(query_4_1,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [8]:
query_4_2 = f'''
    select
	    count(*)
    from 
        public.vacancies
    where salary_from is not null or salary_to is not null /* Одно из полей зарплат не null */
    '''

df = pd.read_sql_query(query_4_2,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [9]:
query_4_3 = f'''
    select
	    round(avg(salary_from)) avg_down, /* Средняя нижняя граница зарплаты */
	    round(avg(salary_to)) avg_up /* Средняя верхняя граница зарплаты */
    from 
        public.vacancies
    '''

df = pd.read_sql_query(query_4_3,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg_down,avg_up
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [10]:
query_4_4 = f'''
    select
	    schedule,
	    employment,
	    count(id) vacancies /* Подсчет числа вакансий */
    from 
        public.vacancies
    group by schedule, employment /* Группировка по рабочему графику и типу трудоустройства */
    order by vacancies desc
    '''

df = pd.read_sql_query(query_4_4,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,schedule,employment,vacancies
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [11]:
query_4_5 = f'''
    select
	    experience,
	    count(id) vacancies /* Подсчет числа вакансий */
    from 
	    public.vacancies
    group by experience /* Группировка по требуемому опыту */
    order by vacancies
    '''

df = pd.read_sql_query(query_4_5,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,vacancies
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

__Выводы по детальному анализу данных:__ В ходе детального анализа видно, что наибольшее количество вакансий ожидаемо представлено в наиболее крупных городах. Также выявлена особенность заполненности данных о зарплатах: более чем у половины вакансий (51%) не заполнено ни одного поля зарплаты из двух. У остальных вакансий в среднем зарплатная вилка составляет от 71065 до 110537 рублей. Наиболее востребованы в подавляющем большинстве работники, готовые работать полный рабочий день при полной занятости, а в отношении требуемого опыта в наибольшем количестве вакансий фигурирует опыт от 1 года до 3 лет.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [12]:
query_5_1 = f'''
    select
	    e.name employer, /* Названия работодателей в таблице employers */
	    count(v.id) vacancies /* Подсчет числа вакансий */
    from 
	    public.vacancies v
	    left join public.employers e on v.employer_id = e.id /* Соединение таблиц вакансий и работодателей */
    group by employer /* Группировка по названиям работодателей */
    order by vacancies desc
    limit 5
    '''

df = pd.read_sql_query(query_5_1,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employer,vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [13]:
query_5_2 = f'''
    select
	    a.name area, /* Названия территорий */
	    count(v.id) vacancies, /* Подсчет числа вакансий */
	    count(e.id) employers /* Подсчет числа работодателей */
    from 
	    public.areas a
	    left join public.vacancies v on v.area_id = a.id /* Присоединеняем к таблице территорий таблицу вакансий */
	    left join public.employers e on e.area = a.id /* Присоединеняем к таблице территорий таблицу работодателей */
    group by a.id
    having count(v.id) = 0 /* Число вакансий должно быть равно нулю */
    order by count(e.id) desc /* Сортируем по количеству работодателей */
    '''

df = pd.read_sql_query(query_5_2,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,vacancies,employers
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
588,Лиман (Астраханская область),0,0
589,Хоринск,0,0
590,Старица,0,0
591,Ессентукская,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [14]:
query_5_3 = f'''
    select
	    e.name, /* Названия работодателей */
	    count(distinct v.area_id) areas /* Подсчет числа терриотрий */
    from 
	    public.vacancies v
	    left join public.employers e on e.id = v.employer_id /* Присоединеняем к таблице вакансий таблицу работодателей */
    group by e.name /* Группировка по названию работодателей */
    order by areas desc
    '''

df = pd.read_sql_query(query_5_3,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [15]:
query_5_4 = f'''
    select
	    count(*) employers /* Подсчет числа работодателей */
    from 
	    public.employers e
	    left join public.employers_industries ei on ei.employer_id = e.id /*  К таблице работодателей таблицу связи работодателей и сфер деятельности */
    where ei.industry_id is null /* Фильтарация пустых значений сфер деятельности */
    '''

df = pd.read_sql_query(query_5_4,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [16]:
query_5_5 = f'''
    select
	    e.name employer, /* Название работодателя */
	    count(industry_id) employers_industries /* Подсчет числа сфер деятельности */
    from 
	    public.employers_industries ei
	    left join public.employers e on ei.employer_id = e.id /* К таблице сфер деятельности и работодателей таблицу работодателей */
    group by employer /* Группировка по названию работодателей */
    having count(industry_id) = 4 /* Фильтрация по количеству сфер деятельности равному 4 */
    order by employer /* Сортировка по названию работодателя */
    limit 1 offset 2 /* Вывод третьего места в списке работодателей */
    '''

df = pd.read_sql_query(query_5_5,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employer,employers_industries
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [17]:
query_5_6 = f'''
    select
	    count(ei.employer_id) employers /* Подсчет числа работодателей */
    from 
	    public.employers_industries ei
	    join public.industries i on ei.industry_id = i.id /* К таблице связи работодателей и сфер деятельности таблицу сфер деятельности */
    group by i.id /* Группировка по сферам деяетельности */
    having i.name = 'Разработка программного обеспечения' /* Фильтрация по нужной сфере деятельности */
    '''

df = pd.read_sql_query(query_5_6,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employers
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [18]:
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'

cities = list() # Cоздадим пустой список городов-миллионников

response = requests.get(url) # Формируем запрос странцицы
page = BeautifulSoup(response.text, 'html.parser') # Получаем HTML-код страницы

result_td = page.find_all('td') # Сначала получаем все участки текста с тегом <td>
for result in result_td:
    result_href = result.find(href=True) # Из полученных результатов оставляем участки с тегом <href>
    if result_href is not None:
        cities.append(result_href.text) # Добавляем результат в список городов непустые значения

# Очищаем список от ненужных элементов:
cities.pop(0) # Удаляем первый элемент, т.к. к городам он не имеет отношения
del cities[16:] # Зная число городов-миллионников, оставляем в списке только это количество

cities_tpl = tuple(cities) # Переводим в кортеж, чтобы передать в SQL-запрос

print(cities_tpl)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [19]:
query_5_7 = f'''
    select
		a.name, /* Названия территорий */
		count(*) cnt /* Подсчет числа вакансий */
	from
		public.vacancies v
		join public.areas a on v.area_id = a.id /* Присоединеняем к таблице вакансий таблицу территорий */
		join public.employers e on v.employer_id = e.id /* Присоединеняем к таблице вакансий таблицу работодателей */
	where e.name = 'Яндекс' /* Фильтрация по нужному работодателю */
	group by a.name /* Группировка по названию территории */
	having a.name in {cities_tpl} /* Фильтрация по сформированному выше кортежу с городами-миллионниками */

	union all /* Объединяем получившуюся таблицу и строку с общим числом вакансий */

	select
		'Total',
		count(*) /* Подсчет числа вакансий */
	from
		public.vacancies v
		join public.areas a on v.area_id = a.id /* Присоединеняем к таблице вакансий таблицу территорий */
		join public.employers e on v.employer_id = e.id /* Присоединеняем к таблице вакансий таблицу работодателей */
	where e.name = 'Яндекс' and a.name in {cities_tpl} /* Фильтрация по нужному работодателю и списку городов */

	order by cnt /* Сортировка по числу вакансий, чтобы Total оказался внизу (как в примере) */
    '''

df = pd.read_sql_query(query_5_7,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

__Выводы по анализу работодателей:__ В результате детального анализа работоделей можно прийти к закономерному выводу, что наибольшее число вакансий предлагают крупнейшие добывающие, IT-компании, банки госудраственного масшатаба, однако по охвату регионов, за исключением "Яндекс", такой закономерности не прослеживается. Больше чем у половины работодателей не указна сфера деяетельности, наиболее вероятно по причине того, что далеко не все сферы деятельности можно охарактеризовать однозначно, однако существуют и такие работодатели, которые охватывают сразу несколько сфер. Также выявлено, что порядка четверти всех работодателей занимаются разработкой программного обеспечения. На примере компнии "Яндекс" также можно сделать закономерный вывод о том, что наибольшее количество вакансий в сфере IT-технологий размещено в наиболее крупных городах, таких как Москва и Санкт-Перербург, а также в городах с населением свыше миллиона человек.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [20]:
query_6_1 = f'''
    select
		count(*) vacancies /* Подсчет числа вакансий */
	from
		public.vacancies
	where 
		name ilike '%data%' or name ilike '%данн%'  /* Фильтрация по наличию в названии вакансии нужных строк */
    '''

df = pd.read_sql_query(query_6_1,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [21]:
query_6_2 = f'''
    select
		count(*) vacancies
	from
		public.vacancies
	where 
		(name ilike '%data scientist%' or name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or name ilike 'исследователь данных%'
		or (name like '%ML%' and name not like '%HTML%')
		or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
		and (name like '%junior%' or name like '%Junior%' /* Фильтрация вакансий Junior */
		or experience = 'Нет опыта'
		or employment = 'Стажировка')
    '''

df = pd.read_sql_query(query_6_2,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [22]:
query_6_3 = f'''
    select
	    count(*) vacancies
    from
	    public.vacancies
    where 
	    (name ilike '%data scientist%' or name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or name ilike 'исследователь данных%'
		or (name like '%ML%' and name not like '%HTML%')
		or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
	    and (key_skills like '%SQL%' or key_skills like '%postgres%') /* Фильтрация наличия SQL и postgres в ключевых навыках */
    '''

df = pd.read_sql_query(query_6_3,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [23]:
query_6_4 = f'''
    select
		count(*) vacancies
	from
		public.vacancies
	where 
		(name ilike '%data scientist%' or name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or name ilike 'исследователь данных%'
		or (name like '%ML%' and name not like '%HTML%')
		or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
		and key_skills like '%Python%' /* Фильтрация наличия Python в ключевых навыках */
    '''

df = pd.read_sql_query(query_6_4,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [24]:
query_6_5 = f'''
    select
		round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) avg_key_skills /* Подсчет среднего числа ключевых навыков */
	from
		public.vacancies
	where 
		(name ilike '%data scientist%' or name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or name ilike 'исследователь данных%'
		or (name like '%ML%' and name not like '%HTML%')
		or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
		and key_skills is not null /* Фильтрация заполненности ключевых навыков */
    '''

df = pd.read_sql_query(query_6_5,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,avg_key_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [25]:
query_6_6 = f'''
    select
		experience, /* Требуемый опыт */
		round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)) avg_salary /* Подсчет средней зарплаты */
	from
		public.vacancies
	where 
		(name ilike '%data scientist%' or name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or name ilike 'исследователь данных%'
		or (name like '%ML%' and name not like '%HTML%')
		or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
		and (salary_from is not null or salary_to is not null) /* Фильтрация заполненности значений зарплат */
	
	group by experience /* Группировка по опыту */
    '''

df = pd.read_sql_query(query_6_6,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

__Выводы по предметному анализу:__ В ходе предметного анализа мы исследовали требования работодателей к дата-сайентистам и вывели ряд количественных показателей, а именно:  
Из всех представленных вакансий к данным имеет отношение 1771 вакансия.   
Из всех вакансий для дата-сайентистов представлена 51 вакансия интересующего нас уровня Junior.  
В роли ключевых навыков для дата-сайентиста Python встречается более чем в полтора раза чаще, чем SQL или postgres (351 вакансии протв 201).  
В среднем же, в вакансиях для DS указывают 6.41 ключевых навыка.   
Дата-сайентист без опыта работы может рассчитывать в среднем на зарплату в 74643 рубля.

# Общий вывод по проекту: 
В ходе исследования сделан вывод о том, что что наибольшее число вакансий вне зависимости от специальности предлагают крупнейшие добывающие, IT-компании, крупенейшие финансовые организации. Наибольшее число вакансий закономерно сосредоточено в наиболее крупных городах, таких как Москва и Санкт-Перербург, а также в городах с населением свыше миллиона человек. Наиболее востребованы в подавляющем большинстве работники, готовые работать полный рабочий день при полной занятости, а в отношении требуемого опыта в наибольшем количестве вакансий фигурирует опыт от 1 года до 3 лет.  
Результаты запросов в ходе предметного анализа позволяют сделать вывод о том, что средняя зарплата начинающего дата-сайентиста без опыта работы (74643 рубля) примерно соответствует нижней границе общей средней зарплаты (от 71065 до 110537 рублей), однако из всех 480 вакансий, касающихся Data Science, только на 51 должность требуется специалист уровня Junior, что свидетельствует высокой конкуренции на этапе поиска первой работы дата-сайентиста.  
В ходе проведения дальнейших исследований можно уделить внимание запросам, отражающим востребованность специалистов Data Science от сферы деятельности предприятий-работодателей, а также географических районов.


# Дополнительные исследования данных

С помощью нижеприведенного запроса получены данные о распределении вакансий дата-сайентистов в зависимости от сферы деятельности работодаетелей. Таким образом, данный запрос позволяет выяснить, в каких отраслях профессия Data Science наиоболее востребована.

In [26]:
query_7_1 = f'''
    select
		i.name industry, /* Название сферы деятельности */
		count(*) vacancies /* Подсчет числа вакансий */
	from 
 		public.vacancies v
		left join public.employers_industries ei on v.employer_id = ei.employer_id /* К таблице вакансий таблицу связи работодателей и сфер деятельности */
		left join public.industries i on ei.industry_id = i.id /* К таблице вакансий таблицу сфер деятельности */
	where 
		(v.name ilike '%data scientist%' or v.name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or v.name ilike 'исследователь данных%'
		or (v.name like '%ML%' and v.name not like '%HTML%')
		or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%')
	group by i.name /* Группировка по названию сферы деятельности */
	having 
 		i.name is not null /* Фильтрация вакансий, где сфера деятельности неизвестна */
	order by vacancies desc
	limit 10
    '''

df = pd.read_sql_query(query_7_1,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,vacancies
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,92
3,"Интернет-компания (поисковики, платежные систе...",79
4,Мобильная связь,23
5,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22
6,"Управляющая, инвестиционная компания (управлен...",21
7,Консалтинговые услуги,16
8,Фиксированная связь,13
9,"Научно-исследовательская, научная, академическ...",9


Следующий запрос создан с личной целью - оценить, сколько вакансий дата-сайентистов уровня Junior предложено в интересующих меня городах Хабаровск и Владивосток. Как оказалось, на момент создания базы данных это были не самые удачные города для начала карьеры.

In [27]:
query_7_2 = f'''
    select
	    v.name, /* Интересующие нас поля */
	    v.key_skills,
	    v.experience,
	    v.salary_from,
	    a.name area, 
	    e.name employer_name
    from public.vacancies v
	    join public.areas a on a.id = v.area_id /* Присоединяем к таблице вакансий таблицу территорий */
	    join public.employers e on e.id = v.employer_id /* Присоединяем к таблице вакансий таблицу работодателей */
    where 
	    (v.name ilike '%data scientist%' or v.name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
	    or v.name ilike 'исследователь данных%'
	    or (v.name like '%ML%' and v.name not like '%HTML%')
	    or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%')
	    and (v.name like '%junior%' or v.name like '%Junior%' /* Фильтрация вакансий Junior */
	    or v.experience = 'Нет опыта'
	    or v.employment = 'Стажировка')
	    and a.name in ('Хабаровск', 'Владивосток') /* Фильтрация по интересующим городам */
    '''

df = pd.read_sql_query(query_7_2,connection)
display(df)

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,key_skills,experience,salary_from,area,employer_name
0,Data Scientist (Computer Vision) / Junior,Python\tLinux\tGit\tSQL\tDocker\tBash\tOpenCV\...,Нет опыта,70000,Владивосток,DNS Головной офис


Результаты предыдущего запроса побудили создать запрос, позволяющий оценить, в каких городах представлено наибольшее число вакансий дата-сайентистов уровня Junior. Результаты оказались ожидаемы: самыми благоприятными городами для новоиспеченных специалистов являются три крупнейших города России.

In [28]:
query_7_3 = f'''
   	select
		a.name area, /* Название територии */
		count(*) vacancies /* Подсчет числа вакансий */
	from public.vacancies v
		join public.areas a on a.id = v.area_id /* Присоединяем к таблице вакансий таблицу территорий */
	where 
		(v.name ilike '%data scientist%' or v.name ilike '%data science%' /* Фильтрация вакансий дата-сайентистов */
		or v.name ilike 'исследователь данных%'
		or (v.name like '%ML%' and v.name not like '%HTML%')
		or v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%')
		and (v.name like '%junior%' or v.name like '%Junior%' /* Фильтрация вакансий Junior */
		or v.experience = 'Нет опыта'
		or v.employment = 'Стажировка')
	group by a.name /* Группировка по городу */
	order by vacancies desc
    '''

df = pd.read_sql_query(query_7_3,connection)
display(df)

connection.close()

c:\Users\Idea pad\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,vacancies
0,Москва,19
1,Санкт-Петербург,9
2,Новосибирск,5
3,Минск,2
4,Воронеж,2
5,Екатеринбург,2
6,Ростов-на-Дону,2
7,Пермь,2
8,Томск,2
9,Казань,2
